BloomTech Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# Read New York City property sales data
import pandas as pd
!pip install pandas-profiling
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [ ]:
# Change column names: replace spaces with underscores
cols = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE']
use_cols = []
for i in cols:
  if " " in i:
    use_cols.append(i.replace(' ','_'))
  else:
    use_cols.append(i)
print(use_cols)

df.columns = use_cols


['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE']


In [ ]:
# Get Pandas Profiling Report
from pandas_profiling import ProfileReport
#ProfileReport(df).to_notebook_iframe()

In [ ]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)
df = df[df['NEIGHBORHOOD']=='TRIBECA']


In [ ]:
# Q. What's the date range of these property sales in Tribeca?
begin,*_,end = df['SALE_DATE']
print(begin,end)

01/03/2019 04/30/2019


In [ ]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers
df['SALE_PRICE'] = df['SALE_PRICE'].str.replace('$','').str.replace('-','').str.replace(',','').astype(int)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Q. What is the maximum SALE_PRICE in this dataset?
maximumsaleprice = df['SALE_PRICE'].max()
maximumsaleprice

260000000

In [ ]:
# Look at the row with the max SALE_PRICE
df[df['SALE_PRICE'] == maximumsaleprice]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,02/01/2019


In [ ]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?
df['TOTAL_UNITS'].value_counts().sum() 
df['TOTAL_UNITS'].value_counts().value_counts()[1:].sum()


2

In [ ]:
# Keep only the single units
df = df[df['TOTAL_UNITS'] == 1]

In [ ]:
# Q. Now what is the max sales price? How many square feet does it have?
df['SALE_PRICE'].max()

39285000

In [ ]:
# Q. How often did $0 sales occur in this subset of the data?
df1 = df[df['SALE_PRICE'] == 0]
df1['SALE_PRICE'].value_counts()
# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 



0    15
Name: SALE_PRICE, dtype: int64

In [ ]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?
df2 = df[df['GROSS_SQUARE_FEET'] > 5000]
df2['GROSS_SQUARE_FEET'].max() 

39567.0

In [ ]:
# What are the building class categories?
# How frequently does each occur?
df2['BUILDING_CLASS_CATEGORY'].unique()
df2['BUILDING_CLASS_CATEGORY'].value_counts()

15 CONDOS - 2-10 UNIT RESIDENTIAL    2
13 CONDOS - ELEVATOR APARTMENTS      1
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [ ]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments
df3 = df[(df['SALE_PRICE']>0) & (df['BUILDING_CLASS_CATEGORY'] == '13 CONDOS - ELEVATOR APARTMENTS')]
# Check how many rows you have now. (Should be 106 rows.)
df3

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,2800000,01/03/2019
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,2650000,01/07/2019
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,1005000,01/09/2019
1543,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,3624,NaN,R4,"2 RIVER TERRACE, 4C",4C,0.0,1.0,0.0,1.0,0,1088.0,2006.0,2,R4,1678000,01/10/2019
1855,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,2610,NaN,R4,"380 RECTOR PLACE, 10A",10A,10280.0,1.0,0.0,1.0,0,873.0,0.0,2,R4,1380000,01/11/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,5761259,04/24/2019
22732,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,2600000,04/29/2019
22733,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,605000,04/29/2019
22897,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,960000,04/30/2019


In [ ]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE
import plotly.express as  px 

fig = px.scatter(df3, x="GROSS_SQUARE_FEET", y="SALE_PRICE",trendline='ols')
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"



In [ ]:
# Look at sales for more than $35 million
df3[df3['SALE_PRICE'] > 35000000]
# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?

#I think its safe to assume that the sale price was a total but as far as the data goes we cant make that assumption without a guarantee 

# Is this dirty data?
# yes, we could divide the total evenly across each supposed sale but we would need a confirmation that that was the case before we made any changes 

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
8370,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1105,NaN,R4,"70 VESTRY STREET, 3C",3C,10013.0,1.0,0.0,1.0,0,1670.0,2016.0,2,R4,36681561,02/12/2019
8371,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1123,NaN,R4,"70 VESTRY STREET, 6C",6C,10013.0,1.0,0.0,1.0,0,1906.0,2016.0,2,R4,36681561,02/12/2019
8372,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1124,NaN,R4,"70 VESTRY STREET, 6D",6D,10013.0,1.0,0.0,1.0,0,2536.0,2016.0,2,R4,36681561,02/12/2019
8373,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1125,NaN,R4,"70 VESTRY STREET, 6E",6E,10013.0,1.0,0.0,1.0,0,2965.0,2016.0,2,R4,36681561,02/12/2019
8374,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1126,NaN,R4,"70 VESTRY STREET, 6F",6F,10013.0,1.0,0.0,1.0,0,2445.0,2016.0,2,R4,36681561,02/12/2019
8375,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1127,NaN,R4,"70 VESTRY STREET, 7A",7A,10013.0,1.0,0.0,1.0,0,2844.0,2016.0,2,R4,36681561,02/12/2019
8376,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1128,NaN,R4,"70 VESTRY STREET, 7B",7B,10013.0,1.0,0.0,1.0,0,3242.0,2016.0,2,R4,36681561,02/12/2019
8377,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1129,NaN,R4,"70 VESTRY STREET, 7C",7C,10013.0,1.0,0.0,1.0,0,1906.0,2016.0,2,R4,36681561,02/12/2019
8378,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1130,NaN,R4,"70 VESTRY STREET, 7D",7D,10013.0,1.0,0.0,1.0,0,2536.0,2016.0,2,R4,36681561,02/12/2019
8379,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1131,NaN,R4,"70 VESTRY STREET, 7E",7E,10013.0,1.0,0.0,1.0,0,2965.0,2016.0,2,R4,36681561,02/12/2019


In [ ]:
# Make a judgment call:
# Keep rows where sale price was < $35 million
df3 = df3[df3['SALE_PRICE'] < 35000000]
# Check how many rows you have now. (Should be down to 90 rows.)
df3

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,2800000,01/03/2019
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,2650000,01/07/2019
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,1005000,01/09/2019
1543,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,3624,NaN,R4,"2 RIVER TERRACE, 4C",4C,0.0,1.0,0.0,1.0,0,1088.0,2006.0,2,R4,1678000,01/10/2019
1855,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,2610,NaN,R4,"380 RECTOR PLACE, 10A",10A,10280.0,1.0,0.0,1.0,0,873.0,0.0,2,R4,1380000,01/11/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,5761259,04/24/2019
22732,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,2600000,04/29/2019
22733,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,605000,04/29/2019
22897,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,960000,04/30/2019


In [ ]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline
fig = px.scatter(df3, x="GROSS_SQUARE_FEET", y="SALE_PRICE",trendline='ols')
fig.show()

In [ ]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
df3.reset_index(inplace=True)
df3.to_csv('tribeca.csv')